In [2]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np
import category_encoders as ce
df = pd.read_csv('preprocessed_dataset.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
df["FIRE_SIZE_CLASS"] = lb_make.fit_transform(df["FIRE_SIZE_CLASS"]) # because it is ordinal
#df["STAT_CAUSE_DESCR"] = lb_make.fit_transform(df["STAT_CAUSE_DESCR"])
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'])
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'])
df['DISCOVERY_DATE']=df.DISCOVERY_DATE.apply(lambda x: x.strftime('%Y%m%d')).astype(int)
df['CONT_DATE'] = df.CONT_DATE.apply(lambda x: x.strftime('%Y%m%d')).astype(int)
df

,FOD_ID,NWCG_REPORTING_AGENCY,SOURCE_REPORTING_UNIT_NAME,FIRE_YEAR,DISCOVERY_DATE,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME,COUNTY
0,46487,FS,Salmon-Challis National Forest,1992,19920620,1.0,Lightning,0.1,0,45.308333,-114.475000,USFS,ID,1035,19920620,1336.0,Lemhi County
1,242377,BLM,Rock Springs Field Office,1992,19920821,1.0,Lightning,0.1,0,41.416600,-109.234000,BLM,WY,1428,19920822,1800.0,Sweetwater County
2,49259,FS,Shasta-Trinity National Forest,1992,19920624,1.0,Lightning,0.1,0,40.730000,-122.475000,USFS,CA,1135,19920624,1800.0,Shasta County
3,50588,FS,Malheur National Forest,1992,19920801,1.0,Lightning,0.1,0,44.296667,-118.738333,USFS,OR,1538,19920801,2200.0,Grant County
4,222205,BLM,Susanville District,1992,19920628,1.0,Lightning,0.1,0,40.666600,-120.484400,BLM,CA,1100,19920628,1324.0,Lassen County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376096,300286438,ST/C&L,Arizona State Forestry Division - State Office,2015,20150316,13.0,Missing/Undefined,0.1,0,33.493680,-112.277619,MISSING/NOT SPECIFIED,AZ,1537,20150316,1800.0,Maricopa County
376097,300287009,ST/C&L,San Mateo-Santa Cruz Unit,2015,20150423,13.0,Missing/Undefined,1.0,1,37.414050,-122.193200,MISSING/NOT SPECIFIED,CA,1321,20150423,1334.0,SAN MATEO
376098,300307593,ST/C&L,Butte Unit,2015,20150812,13.0,Missing/Undefined,0.1,0,39.501869,-121.605211,MISSING/NOT SPECIFIED,CA,1246,20150812,1300.0,BUTTE
376099,300288242,ST/C&L,Arizona State Forestry Division - State Office,2015,20150711,13.0,Missing/Undefined,0.1,0,32.230100,-110.798700,MISSING/NOT SPECIFIED,AZ,1382,20150711,1800.0,Pima County


In [4]:
X = df.drop(columns=['STAT_CAUSE_CODE','STAT_CAUSE_DESCR','OWNER_DESCR','FOD_ID','NWCG_REPORTING_AGENCY',
                    'CONT_TIME','FIRE_SIZE_CLASS','SOURCE_REPORTING_UNIT_NAME'])
#y = df['STAT_CAUSE_DESCR']
y = df['STAT_CAUSE_CODE'].astype(int)

In [5]:
#features= ['NWCG_REPORTING_AGENCY','SOURCE_REPORTING_UNIT_NAME','STATE','COUNTY']
features= ['STATE','COUNTY']
encoder_stat_county = ce.TargetEncoder(cols = features,drop_invariant=True, return_df=True)
X = encoder_stat_county.fit_transform(X,y)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled,columns=X.columns.tolist())
X

,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,CONT_DATE,COUNTY
0,-1.757433,-1.757325,-0.030397,1.392022,-1.123735,-1.199943,-1.347937,-1.757356,-2.367189
1,-1.757433,-1.754308,-0.030397,0.756642,-0.809814,-0.914533,-0.069827,-1.754325,-2.008904
2,-1.757433,-1.757265,-0.030397,0.644545,-1.602914,-0.182842,-1.022718,-1.757296,-0.485467
3,-1.757433,-1.754608,-0.030397,1.226853,-1.379097,-1.902139,0.287914,-1.754640,-1.648891
4,-1.757433,-1.757205,-0.030397,0.634194,-1.483682,-0.182842,-1.136545,-1.757236,-1.683963
...,...,...,...,...,...,...,...,...,...
376096,1.694389,1.690056,-0.030397,-0.536885,-0.992118,-0.766288,0.284662,1.689878,1.447053
376097,1.694389,1.691662,-0.030018,0.103170,-1.586034,-0.182842,-0.417811,1.691484,1.891358
376098,1.694389,1.697501,-0.030397,0.444035,-1.550816,-0.182842,-0.661725,1.697322,0.941119
376099,1.694389,1.695985,-0.030397,-0.743182,-0.903535,-0.766288,-0.219428,1.695806,0.616668


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=True,random_state=15,test_size=0.2)

In [8]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train, y_train) 
gnb_predictions = gnb.predict(X_test) 
  
# accuracy on X_test 
accuracy = gnb.score(X_test, y_test) 
print(accuracy) 

0.32758139349383814


In [9]:
from sklearn.model_selection import cross_val_score
clf = GaussianNB()
scores = cross_val_score(clf, X, y, cv=10)
print("Accuracy : {:0.2f} (+/- {:0.2f})".format(scores.mean(), scores.std()))

Accuracy : 0.32 (+/- 0.08)


In [59]:
import pandas as pd
import numpy as np
import category_encoders as ce
training_df = pd.read_csv('training_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

FileNotFoundError: [Errno 2] File training_dataset.csv does not exist: 'training_dataset.csv'

In [49]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
df["FIRE_SIZE_CLASS"] = lb_make.fit_transform(df["FIRE_SIZE_CLASS"]) # because it is ordinal
df["STAT_CAUSE_DESCR"] = lb_make.fit_transform(df["STAT_CAUSE_DESCR"]) # because it is ordinal
#df["STAT_CAUSE_DESCR"] = lb_make.fit_transform(df["STAT_CAUSE_DESCR"])
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'])
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'])
df['DISCOVERY_DATE']=df.DISCOVERY_DATE.apply(lambda x: x.strftime('%Y%m%d')).astype(int)
df['CONT_DATE'] = df.CONT_DATE.apply(lambda x: x.strftime('%Y%m%d')).astype(int)
df

,FOD_ID,NWCG_REPORTING_AGENCY,SOURCE_REPORTING_UNIT_NAME,FIRE_YEAR,DISCOVERY_DATE,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME,COUNTY
0,46487,FS,Salmon-Challis National Forest,1992,19920620,1.0,6,0.1,0,45.308333,-114.475000,USFS,ID,1035,19920620,1336.0,Lemhi County
1,242377,BLM,Rock Springs Field Office,1992,19920821,1.0,6,0.1,0,41.416600,-109.234000,BLM,WY,1428,19920822,1800.0,Sweetwater County
2,49259,FS,Shasta-Trinity National Forest,1992,19920624,1.0,6,0.1,0,40.730000,-122.475000,USFS,CA,1135,19920624,1800.0,Shasta County
3,50588,FS,Malheur National Forest,1992,19920801,1.0,6,0.1,0,44.296667,-118.738333,USFS,OR,1538,19920801,2200.0,Grant County
4,222205,BLM,Susanville District,1992,19920628,1.0,6,0.1,0,40.666600,-120.484400,BLM,CA,1100,19920628,1324.0,Lassen County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376096,300286438,ST/C&L,Arizona State Forestry Division - State Office,2015,20150316,13.0,8,0.1,0,33.493680,-112.277619,MISSING/NOT SPECIFIED,AZ,1537,20150316,1800.0,Maricopa County
376097,300287009,ST/C&L,San Mateo-Santa Cruz Unit,2015,20150423,13.0,8,1.0,1,37.414050,-122.193200,MISSING/NOT SPECIFIED,CA,1321,20150423,1334.0,SAN MATEO
376098,300307593,ST/C&L,Butte Unit,2015,20150812,13.0,8,0.1,0,39.501869,-121.605211,MISSING/NOT SPECIFIED,CA,1246,20150812,1300.0,BUTTE
376099,300288242,ST/C&L,Arizona State Forestry Division - State Office,2015,20150711,13.0,8,0.1,0,32.230100,-110.798700,MISSING/NOT SPECIFIED,AZ,1382,20150711,1800.0,Pima County


In [50]:
X = df.drop(columns=['STAT_CAUSE_CODE','STAT_CAUSE_DESCR','OWNER_DESCR','FOD_ID','DISCOVERY_TIME',
                    'CONT_TIME','FIRE_SIZE_CLASS','SOURCE_REPORTING_UNIT_NAME','NWCG_REPORTING_AGENCY'])
#y = df['STAT_CAUSE_DESCR']
y = df['STAT_CAUSE_DESCR']

In [51]:
#features= ['NWCG_REPORTING_AGENCY','SOURCE_REPORTING_UNIT_NAME','STATE','COUNTY']
features= ['STATE','COUNTY']
encoder_stat_county = ce.TargetEncoder(cols = features,drop_invariant=True, return_df=True)
X = encoder_stat_county.fit_transform(X,y)

In [52]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=True,random_state=15,test_size=0.2)

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled,columns=X.columns.tolist())
X_train

,FIRE_YEAR,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,CONT_DATE,COUNTY
0,-0.256530,-0.258119,-0.023809,-0.967023,0.777899,-1.213035,-0.258220,-0.079361
1,-0.556654,-0.562650,-0.029468,-0.996370,0.480318,-1.461538,-0.562738,-2.389502
2,-0.556654,-0.559483,-0.029106,-0.267877,1.032450,-0.254639,-0.559571,-0.108667
3,0.943964,0.940901,-0.029428,1.506454,0.371275,-0.157382,0.940748,-0.709174
4,-0.406592,-0.397276,-0.029187,-0.661421,0.929529,0.015176,-0.397371,-0.474539
...,...,...,...,...,...,...,...,...
300875,0.343717,0.348452,-0.029468,1.843075,-1.544974,0.460505,0.348325,-0.091598
300876,1.094026,1.094119,-0.029106,-1.142442,-0.093586,0.124710,1.093960,-0.755244
300877,-1.006839,-1.009954,-0.026699,-0.624038,0.950646,0.015176,-1.010008,-0.254505
300878,0.943964,0.939760,-0.029307,1.807459,0.042330,-1.159892,0.939608,-1.095686


In [54]:
X_test_scaled = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test_scaled,columns=X.columns.tolist())

In [34]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train[:, np.newaxis]).toarray()
y_test = ohe.fit_transform(y_test[:, np.newaxis]).toarray()

In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
model = LogisticRegression(solver = 'lbfgs')
model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [57]:
# use the model to make predictions with the test data
y_pred = model.predict(X_test)
# how did our model perform?
count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = model.score(X_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 46565
Accuracy: 1.00


In [58]:
model.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])